<img src="https://www.unir.net/wp-content/uploads/2019/11/Unir_2021_logo.svg" width="240" height="240" align="right"/>

<center><h1>TRABAJO FIN DE MÁSTER</header1></center>

## Importación de librerias necesarias

In [109]:
#Para esta actividad se importarán las siguientes librerías:
from pandas import read_csv
import numpy as np

Fichero 0: Maestra de municipios (contiene los 183 municipios que vamos a analizar, con el fin de poder cruzar con todos los ficheros.

In [8]:
import pandas as pd

ruta = r"C:\Users\elena\Desktop\universidad\Máster Big Data\Trabajo Fin de Máster\Datos\DIM_MUNICIPIOS.xlsx"

municipios = pd.read_excel(ruta)

municipios["COD_MUNICIPIO"] = (
    municipios["MUNICIPIO"]
    .astype(str)
    .str.strip()
    .str[:5]
)

municipios.head(200)

,MUNICIPIO,COD_MUNICIPIO
0,42001 ABEJAR,42001
1,42003 ADRADAS,42003
2,42004 AGREDA,42004
3,42006 ALCONABA,42006
4,42007 ALCUBILLA DE AVELLANEDA,42007
...,...,...
178,42215 VINUESA,42215
179,42216 VIZMANOS,42216
180,42217 VOZMEDIANO,42217
181,42218 YANGUAS,42218


FICHERO 1: ALOJAMIENTOS HOTELEROS Y DE TURISMO RURAL

In [10]:
import pandas as pd

ruta = r"C:\Users\elena\Desktop\universidad\Máster Big Data\Trabajo Fin de Máster\Datos\Alojamientos hoteleros y de turismo rural.csv"

alojamientos = pd.read_csv(ruta,sep=',', encoding='latin1',skiprows=5)

alojamientos.head()

,FECHA,MUNICIPIO,Sum,Sum.1,Unnamed: 4
0,2000,05002 ADRADA (LA),1,1,NaN
1,NaN,05013 ARENAL (EL),1,.,NaN
2,NaN,05014 ARENAS DE SAN PEDRO,6,4,NaN
3,NaN,05016 AREVALO,4,.,NaN
4,NaN,05019 AVILA,34,1,NaN


Durante el preprocesamiento de los datos se realizó la renombración semántica de las variables de interés y la eliminación de columnas redundantes, con el objetivo de mejorar la interpretabilidad del conjunto de datos y facilitar su posterior análisis

In [12]:
# la quinta columna sobra, ponemos los encabezados

alojamientos.drop(columns=alojamientos.columns[-1], inplace=True)

alojamientos.rename(
    columns={
        alojamientos.columns[2]: "ALOJAMIENTOS_HOTELEROS",
        alojamientos.columns[3]: "ALOJAMIENTOS_TURISMO_RURAL"
    },
    inplace=True
)

cols = [
    "ALOJAMIENTOS_HOTELEROS",
    "ALOJAMIENTOS_TURISMO_RURAL"
]

alojamientos[cols] = (
    alojamientos[cols]
    .astype(str)
    .apply(lambda x: x.str.strip())
    .replace(".", 0)
    .apply(pd.to_numeric)
)

#Arreglamos la fecha para que aparezca en todas las filas 

alojamientos["FECHA"] = alojamientos["FECHA"].ffill()

# Nos quedamos con los municipios de Soria, que son los que su codigo postal empieza por 42

alojamientos["COD_MUNICIPIO"] = (
    alojamientos["MUNICIPIO"]
    .astype(str)
    .str.strip()
    .str[:5]
)
alojamientos = alojamientos[
    alojamientos["COD_MUNICIPIO"].str.startswith("42")
]

alojamientos.head()

,FECHA,MUNICIPIO,ALOJAMIENTOS_HOTELEROS,ALOJAMIENTOS_TURISMO_RURAL,COD_MUNICIPIO
511,2000,42001 ABEJAR,1,1,42001
512,2000,42004 AGREDA,2,0,42004
513,2000,42019 ALMARZA,2,0,42019
514,2000,42020 ALMAZAN,7,1,42020
515,2000,42025 ARCOS DE JALON,5,0,42025


In [13]:
#los municipios que no aparecen aqui pero cuyo año sí aparece, es porque no tienen ningun alojamiento. 
#para evitar en el modelo confundir un valor NA con lo que deberia ser un cero, vamos a cruzar con la maestra
#de municipios y asignamos los ceros correspondientes

#lo primero que tenemos en cuenta para cruzar son las fechas de alojamientos
anios_existentes = alojamientos["FECHA"].drop_duplicates()


panel_parcial = (
    municipios
    .assign(key=1)
    .merge(anios_existentes.to_frame(name="FECHA").assign(key=1), on="key")
    .drop("key", axis=1)
)

alojamientos = panel_parcial.merge(
    alojamientos,
    on=["FECHA", "COD_MUNICIPIO", "MUNICIPIO"],
    how="left"
)

alojamientos[cols] = alojamientos[cols].apply(pd.to_numeric,errors="coerce").fillna(0)

In [14]:
alojamientos.head()

,MUNICIPIO,COD_MUNICIPIO,FECHA,ALOJAMIENTOS_HOTELEROS,ALOJAMIENTOS_TURISMO_RURAL
0,42001 ABEJAR,42001,2000,1.0,1.0
1,42001 ABEJAR,42001,2001,1.0,2.0
2,42001 ABEJAR,42001,2002,0.0,2.0
3,42001 ABEJAR,42001,2003,0.0,2.0
4,42001 ABEJAR,42001,2004,0.0,5.0


FICHERO 2: CONSULTORIOS

In [16]:
import pandas as pd

ruta = r"C:\Users\elena\Desktop\universidad\Máster Big Data\Trabajo Fin de Máster\Datos\Consultorios.csv"

consultorios = pd.read_csv(ruta,sep=',', encoding='latin1',skiprows=5)

consultorios.head()

,FECHA,MUNICIPIO,Sum,Unnamed: 3
0,2000,05001 ADANERO,1,NaN
1,NaN,05002 ADRADA (LA),1,NaN
2,NaN,05005 ALBORNOS,1,NaN
3,NaN,05007 ALDEANUEVA DE SANTA CRUZ,1,NaN
4,NaN,05008 ALDEASECA,1,NaN


In [17]:
consultorios.drop(columns=consultorios.columns[-1], inplace=True)

consultorios.rename(
    columns={
        consultorios.columns[2]: "CONSULTORIOS",
    },
    inplace=True
)
consultorios.head()

,FECHA,MUNICIPIO,CONSULTORIOS
0,2000,05001 ADANERO,1
1,NaN,05002 ADRADA (LA),1
2,NaN,05005 ALBORNOS,1
3,NaN,05007 ALDEANUEVA DE SANTA CRUZ,1
4,NaN,05008 ALDEASECA,1


In [18]:
cols = [
    "CONSULTORIOS"
]

consultorios[cols] = (
    consultorios[cols]
    .astype(str)
    .apply(lambda x: x.str.strip())
    .replace(".", 0)
    .apply(pd.to_numeric)
)

#Arreglamos la fecha para que aparezca en todas las filas 

consultorios["FECHA"] = consultorios["FECHA"].ffill()

# Nos quedamos con los municipios de Soria, que son los que su codigo postal empieza por 42

consultorios["COD_MUNICIPIO"] = (
    consultorios["MUNICIPIO"]
    .astype(str)
    .str.strip()
    .str[:5]
)
consultorios = consultorios[
    consultorios["COD_MUNICIPIO"].str.startswith("42")
]

consultorios.head()

,FECHA,MUNICIPIO,CONSULTORIOS,COD_MUNICIPIO
1491,2000,42001 ABEJAR,1,42001
1492,2000,42003 ADRADAS,2,42003
1493,2000,42004 AGREDA,1,42004
1494,2000,42006 ALCONABA,3,42006
1495,2000,42007 ALCUBILLA DE AVELLANEDA,3,42007


In [19]:
#los municipios que no aparecen aqui pero cuyo año sí aparece, es porque no tienen ningun consultorio. 
#para evitar en el modelo confundir un valor NA con lo que deberia ser un cero, vamos a cruzar con la maestra
#de municipios y asignamos los ceros correspondientes

#lo primero que tenemos en cuenta para cruzar son las fechas de alojamientos
anios_existentes = consultorios["FECHA"].drop_duplicates()


panel_parcial = (
    municipios
    .assign(key=1)
    .merge(anios_existentes.to_frame(name="FECHA").assign(key=1), on="key")
    .drop("key", axis=1)
)

consultorios = panel_parcial.merge(
    consultorios,
    on=["FECHA", "COD_MUNICIPIO", "MUNICIPIO"],
    how="left"
)

consultorios[cols] = consultorios[cols].apply(pd.to_numeric,errors="coerce").fillna(0)

In [20]:
consultorios.head()


,MUNICIPIO,COD_MUNICIPIO,FECHA,CONSULTORIOS
0,42001 ABEJAR,42001,2000,1.0
1,42001 ABEJAR,42001,2004,1.0
2,42001 ABEJAR,42001,2007,1.0
3,42001 ABEJAR,42001,2009,1.0
4,42001 ABEJAR,42001,2010,1.0


FICHERO 3: FARMACIAS

In [22]:
import pandas as pd

ruta = r"C:\Users\elena\Desktop\universidad\Máster Big Data\Trabajo Fin de Máster\Datos\Farmacias.csv"

farmacias = pd.read_csv(ruta,sep=',', encoding='latin1',skiprows=5)

farmacias.drop(columns=farmacias.columns[-1], inplace=True)

farmacias.head()

,FECHA,MUNICIPIO,Sum
0,2000,05001 ADANERO,1
1,NaN,05002 ADRADA (LA),1
2,NaN,05008 ALDEASECA,1
3,NaN,05010 ALDEHUELA (LA),1
4,NaN,05013 ARENAL (EL),1


In [23]:
farmacias.rename(
    columns={
        farmacias.columns[2]: "FARMACIAS",
    },
    inplace=True
)
farmacias.head()

,FECHA,MUNICIPIO,FARMACIAS
0,2000,05001 ADANERO,1
1,NaN,05002 ADRADA (LA),1
2,NaN,05008 ALDEASECA,1
3,NaN,05010 ALDEHUELA (LA),1
4,NaN,05013 ARENAL (EL),1


In [24]:
cols = [
    "FARMACIAS"
]

farmacias[cols] = (
    farmacias[cols]
    .astype(str)
    .apply(lambda x: x.str.strip())
    .replace(".", 0)
    .apply(pd.to_numeric)
)

#Arreglamos la fecha para que aparezca en todas las filas 

farmacias["FECHA"] = farmacias["FECHA"].ffill()

# Nos quedamos con los municipios de Soria, que son los que su codigo postal empieza por 42

farmacias["COD_MUNICIPIO"] = (
    farmacias["MUNICIPIO"]
    .astype(str)
    .str.strip()
    .str[:5]
)
farmacias = farmacias[
    farmacias["COD_MUNICIPIO"].str.startswith("42")
]


In [25]:
#los municipios que no aparecen aqui pero cuyo año sí aparece, es porque no tienen ninguna farmacia. 
#para evitar en el modelo confundir un valor NA con lo que deberia ser un cero, vamos a cruzar con la maestra
#de municipios y asignamos los ceros correspondientes

#lo primero que tenemos en cuenta para cruzar son las fechas de alojamientos
anios_existentes = farmacias["FECHA"].drop_duplicates()


panel_parcial = (
    municipios
    .assign(key=1)
    .merge(anios_existentes.to_frame(name="FECHA").assign(key=1), on="key")
    .drop("key", axis=1)
)

farmacias = panel_parcial.merge(
    farmacias,
    on=["FECHA", "COD_MUNICIPIO", "MUNICIPIO"],
    how="left"
)

farmacias[cols] = farmacias[cols].apply(pd.to_numeric,errors="coerce").fillna(0)

farmacias.head()

,MUNICIPIO,COD_MUNICIPIO,FECHA,FARMACIAS
0,42001 ABEJAR,42001,2000,1.0
1,42001 ABEJAR,42001,2002,1.0
2,42001 ABEJAR,42001,2004,1.0
3,42001 ABEJAR,42001,2011,1.0
4,42001 ABEJAR,42001,2012,1.0


FICHERO 4: DEMANDA DE EMPLEO

In [27]:
import pandas as pd

ruta = r"C:\Users\elena\Desktop\universidad\Máster Big Data\Trabajo Fin de Máster\Datos\Demanda de empleo.csv"

empleo = pd.read_csv(ruta,sep=',', encoding='latin1',skiprows=5)

empleo.drop(columns=empleo.columns[-1], inplace=True)

empleo.head()

,FECHA,MUNICIPIO,Sum,Sum.1,Sum.2,Sum.3
0,2000,05001 ADANERO,.,7,1,5
1,NaN,05002 ADRADA (LA),13,41,14,48
2,NaN,05005 ALBORNOS,3,8,2,3
3,NaN,05007 ALDEANUEVA DE SANTA CRUZ,.,5,.,1
4,NaN,05008 ALDEASECA,2,19,1,11


In [28]:
empleo.rename(
    columns={
        empleo.columns[2]: "DEMANDAS_MUJERES_MENOR_25",
        empleo.columns[3]: "DEMANDAS_MUJERES_MAYOR_25",
        empleo.columns[4]: "DEMANDAS_VARONES_MENOR_25",
        empleo.columns[5]: "DEMANDAS_VARONES_MAYOR_25",
    },
    inplace=True
)
empleo.head()

,FECHA,MUNICIPIO,DEMANDAS_MUJERES_MENOR_25,DEMANDAS_MUJERES_MAYOR_25,DEMANDAS_VARONES_MENOR_25,DEMANDAS_VARONES_MAYOR_25
0,2000,05001 ADANERO,.,7,1,5
1,NaN,05002 ADRADA (LA),13,41,14,48
2,NaN,05005 ALBORNOS,3,8,2,3
3,NaN,05007 ALDEANUEVA DE SANTA CRUZ,.,5,.,1
4,NaN,05008 ALDEASECA,2,19,1,11


In [29]:
cols = [
    "DEMANDAS_MUJERES_MENOR_25",
    "DEMANDAS_MUJERES_MAYOR_25",
    "DEMANDAS_VARONES_MENOR_25",
    "DEMANDAS_VARONES_MAYOR_25",
]

empleo[cols] = (
    empleo[cols]
    .astype(str)
    .apply(lambda x: x.str.strip())
    .replace(".", 0)
    .apply(pd.to_numeric)
)

#Arreglamos la fecha para que aparezca en todas las filas 

empleo["FECHA"] = empleo["FECHA"].ffill()

# Nos quedamos con los municipios de Soria, que son los que su codigo postal empieza por 42

empleo["COD_MUNICIPIO"] = (
    empleo["MUNICIPIO"]
    .astype(str)
    .str.strip()
    .str[:5]
)
empleo = empleo[
    empleo["COD_MUNICIPIO"].str.startswith("42")
]


In [30]:
#los municipios que no aparecen aqui pero cuyo año sí aparece, es porque no tienen ninguna demanda de empleo. 
#para evitar en el modelo confundir un valor NA con lo que deberia ser un cero, vamos a cruzar con la maestra
#de municipios y asignamos los ceros correspondientes

#lo primero que tenemos en cuenta para cruzar son las fechas de alojamientos
anios_existentes = empleo["FECHA"].drop_duplicates()


panel_parcial = (
    municipios
    .assign(key=1)
    .merge(anios_existentes.to_frame(name="FECHA").assign(key=1), on="key")
    .drop("key", axis=1)
)

empleo = panel_parcial.merge(
    empleo,
    on=["FECHA", "COD_MUNICIPIO", "MUNICIPIO"],
    how="left"
)

empleo[cols] = empleo[cols].apply(pd.to_numeric,errors="coerce").fillna(0)

empleo.head()

,MUNICIPIO,COD_MUNICIPIO,FECHA,DEMANDAS_MUJERES_MENOR_25,DEMANDAS_MUJERES_MAYOR_25,DEMANDAS_VARONES_MENOR_25,DEMANDAS_VARONES_MAYOR_25
0,42001 ABEJAR,42001,2000,0.0,10.0,1.0,3.0
1,42001 ABEJAR,42001,2001,1.0,8.0,1.0,3.0
2,42001 ABEJAR,42001,2002,2.0,5.0,1.0,3.0
3,42001 ABEJAR,42001,2003,0.0,5.0,0.0,4.0
4,42001 ABEJAR,42001,2004,1.0,4.0,3.0,5.0


FICHERO 5: DISTANCIA A LA CAPITAL

In [32]:
import pandas as pd

ruta = r"C:\Users\elena\Desktop\universidad\Máster Big Data\Trabajo Fin de Máster\Datos\distancia a la capital.xlsx"

distancia = pd.read_excel(ruta)

distancia["COD_MUNICIPIO"] = (
    distancia["MUNICIPIO"]
    .astype(str)
    .str.strip()
    .str[:5]
)

#borramos la columna al tener ya el cod municipio
distancia=distancia.drop(columns=["MUNICIPIO"])

In [33]:
distancia.head()

,distancia en km a la capital (soria),COD_MUNICIPIO
0,27.4,42001
1,54.3,42003
2,57.1,42004
3,13.0,42006
4,78.9,42007


FICHERO 6: NUMERO DE EMPRESAS POR ACTIVIDAD PRINCIPAL

In [35]:
import pandas as pd

ruta = r"C:\Users\elena\Desktop\universidad\Máster Big Data\Trabajo Fin de Máster\Datos\Empresas.xlsx"

empresas_raw = pd.read_excel(
    ruta,
    sheet_name=0,
    header=None
)

empresas_raw.head(15)

,0,1,2,3,4,5,6,7,8,9,...,134,135,136,137,138,139,140,141,142,143
0,,Total,Total,Total,Total,Total,Total,Total,Total,Total,...,Total servicios,Total servicios,Total servicios,Total servicios,Total servicios,Total servicios,Total servicios,Total servicios,Total servicios,Total servicios
1,,2024,2023,2022,2021,2020,2019,2018,2017,2016,...,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012
2,42001 Abejar,23,22,18,19,19,20,20,21,20,...,,,,,,,,,,
3,42003 Adradas,8,5,6,6,6,6,6,6,6,...,,,,,,,,,,
4,42004 Ágreda,186,184,188,194,192,200,194,191,197,...,46,47,54,46,45,55,51,48,50,45
5,42006 Alconaba,11,11,9,8,7,5,5,5,8,...,,,,,,,,,,
6,42007 Alcubilla de Avellaneda,5,0,0,0,0,0,0,4,4,...,,,,,,,,,,
7,42008 Alcubilla de las Peñas,8,9,10,10,10,11,11,10,10,...,,,,,,,,,,
8,42009 Aldealafuente,4,0,4,5,4,4,4,4,0,...,,,,,,,,,,
9,42010 Aldealices,0,0,0,0,0,0,0,0,0,...,,,,,,,,,,


In [36]:
fila_tipo = empresas_raw.iloc[0]
fila_fecha = empresas_raw.iloc[1]
datos = empresas_raw.iloc[2:].reset_index(drop=True)

In [37]:
tipos = fila_tipo.iloc[1:].astype(str).str.strip()
fechas = fila_fecha.iloc[1:].astype(str).str.strip()

In [38]:
datos = datos.rename(columns={0: "MUNICIPIO"})

In [39]:
registros = []

for i, col in enumerate(datos.columns[1:], start=1):
    registros.append(
        pd.DataFrame({
            "FECHA": fechas.iloc[i-1],
            "MUNICIPIO": datos["MUNICIPIO"],
            "TIPO_EMPRESA": tipos.iloc[i-1],
            "EMPRESAS": datos[col]
        })
    )

empresas = pd.concat(registros, ignore_index=True)

empresas["COD_MUNICIPIO"] = (
    empresas["MUNICIPIO"]
    .astype(str)
    .str.strip()
    .str[:5]
)
# Nos quedamos solo con el total de empresas
empresas = empresas[
    empresas["TIPO_EMPRESA"].str.strip() == "Total"
]

# Eliminamos la columna TIPO_EMPRESA
empresas.drop(columns=["TIPO_EMPRESA"], inplace=True)

empresas.head(100)

#aqui no hace falta hacer el ajuste con la maestra de municipios ya que ya estan los 183 

,FECHA,MUNICIPIO,EMPRESAS,COD_MUNICIPIO
0,2024,42001 Abejar,23,42001
1,2024,42003 Adradas,8,42003
2,2024,42004 Ágreda,186,42004
3,2024,42006 Alconaba,11,42006
4,2024,42007 Alcubilla de Avellaneda,5,42007
...,...,...,...,...
95,2024,42111 Matamala de Almazán,13,42111
96,2024,42113 Medinaceli,71,42113
97,2024,42115 Miño de Medinaceli,7,42115
98,2024,42116 Miño de San Esteban,4,42116


FICHERO 7: FAMILIAS QUE TRABAJAN DE EXPLOTACIONES AGRARIAS

In [41]:
import pandas as pd

ruta = r"C:\Users\elena\Desktop\universidad\Máster Big Data\Trabajo Fin de Máster\Datos\Familiares que trabajan de explotaciones agragrias.csv"

familias_agricultoras = pd.read_csv(ruta,sep=',', encoding='latin1',skiprows=5)

familias_agricultoras.drop(columns=familias_agricultoras.columns[-1], inplace=True)

familias_agricultoras.head()

,FECHA,MUNICIPIO,Sum
0,1989,05001 ADANERO,72
1,NaN,05002 ADRADA (LA),421
2,NaN,05005 ALBORNOS,100
3,NaN,05007 ALDEANUEVA DE SANTA CRUZ,100
4,NaN,05008 ALDEASECA,113


In [42]:
familias_agricultoras.rename(
    columns={
        familias_agricultoras.columns[2]: "FAMILIARES",
    },
    inplace=True
)
familias_agricultoras.head()

,FECHA,MUNICIPIO,FAMILIARES
0,1989,05001 ADANERO,72
1,NaN,05002 ADRADA (LA),421
2,NaN,05005 ALBORNOS,100
3,NaN,05007 ALDEANUEVA DE SANTA CRUZ,100
4,NaN,05008 ALDEASECA,113


In [43]:
cols = [
    "FAMILIARES"
]

familias_agricultoras[cols] = (
    familias_agricultoras[cols]
    .astype(str)
    .apply(lambda x: x.str.strip())
    .replace(".", 0)
    .apply(pd.to_numeric)
)

#Arreglamos la fecha para que aparezca en todas las filas 

familias_agricultoras["FECHA"] = familias_agricultoras["FECHA"].ffill()

# Nos quedamos con los municipios de Soria, que son los que su codigo postal empieza por 42

familias_agricultoras["COD_MUNICIPIO"] = (
    familias_agricultoras["MUNICIPIO"]
    .astype(str)
    .str.strip()
    .str[:5]
)
familias_agricultoras = familias_agricultoras[
    familias_agricultoras["COD_MUNICIPIO"].str.startswith("42")
]


In [44]:
#los municipios que no aparecen aqui pero cuyo año sí aparece, es porque no tienen ninguna familia dedicada a exp agraria. 
#para evitar en el modelo confundir un valor NA con lo que deberia ser un cero, vamos a cruzar con la maestra
#de municipios y asignamos los ceros correspondientes

#lo primero que tenemos en cuenta para cruzar son las fechas de alojamientos
anios_existentes = familias_agricultoras["FECHA"].drop_duplicates()


panel_parcial = (
    municipios
    .assign(key=1)
    .merge(anios_existentes.to_frame(name="FECHA").assign(key=1), on="key")
    .drop("key", axis=1)
)

familias_agricultoras = panel_parcial.merge(
    familias_agricultoras,
    on=["FECHA", "COD_MUNICIPIO", "MUNICIPIO"],
    how="left"
)

familias_agricultoras[cols] = familias_agricultoras[cols].apply(pd.to_numeric,errors="coerce").fillna(0)

familias_agricultoras.head()

,MUNICIPIO,COD_MUNICIPIO,FECHA,FAMILIARES
0,42001 ABEJAR,42001,1989,55
1,42001 ABEJAR,42001,1999,17
2,42001 ABEJAR,42001,TOTAL,72
3,42003 ADRADAS,42003,1989,33
4,42003 ADRADAS,42003,1999,39


FICHERO 8: HOSPITALES Y CAMAS HOSPITALARIAS

In [46]:
import pandas as pd

ruta = r"C:\Users\elena\Desktop\universidad\Máster Big Data\Trabajo Fin de Máster\Datos\Hospitales y camas hospitalarias.csv"

hospitales = pd.read_csv(ruta,sep=',', encoding='latin1',skiprows=5)

hospitales.drop(columns=hospitales.columns[-1], inplace=True)

hospitales.head()

,FECHA,MUNICIPIO,Sum,Sum.1
0,2000,05016 AREVALO,1,293
1,NaN,05019 AVILA,1,352
2,NaN,09018 ARANDA DE DUERO,2,233
3,NaN,09059 BURGOS,6,1321
4,NaN,09219 MIRANDA DE EBRO,1,126


In [47]:
hospitales.rename(
    columns={
        hospitales.columns[2]: "HOSPITALES",
        hospitales.columns[3]: "CAMAS_HOSPITALARIAS"
    },
    inplace=True
)
hospitales.head()

,FECHA,MUNICIPIO,HOSPITALES,CAMAS_HOSPITALARIAS
0,2000,05016 AREVALO,1,293
1,NaN,05019 AVILA,1,352
2,NaN,09018 ARANDA DE DUERO,2,233
3,NaN,09059 BURGOS,6,1321
4,NaN,09219 MIRANDA DE EBRO,1,126


In [48]:
cols = [
    "HOSPITALES",
    "CAMAS_HOSPITALARIAS"
]

hospitales[cols] = (
    hospitales[cols]
    .astype(str)
    .apply(lambda x: x.str.strip())
    .replace(".", 0)
    .apply(pd.to_numeric)
)

#Arreglamos la fecha para que aparezca en todas las filas 

hospitales["FECHA"] = hospitales["FECHA"].ffill()

# Nos quedamos con los municipios de Soria, que son los que su codigo postal empieza por 42

hospitales["COD_MUNICIPIO"] = (
    hospitales["MUNICIPIO"]
    .astype(str)
    .str.strip()
    .str[:5]
)
hospitales = hospitales[
    hospitales["COD_MUNICIPIO"].str.startswith("42")
]

In [49]:
#los municipios que no aparecen aqui pero cuyo año sí aparece, es porque no tienen ningun hospital
#para evitar en el modelo confundir un valor NA con lo que deberia ser un cero, vamos a cruzar con la maestra
#de municipios y asignamos los ceros correspondientes

#lo primero que tenemos en cuenta para cruzar son las fechas de alojamientos
anios_existentes = hospitales["FECHA"].drop_duplicates()


panel_parcial = (
    municipios
    .assign(key=1)
    .merge(anios_existentes.to_frame(name="FECHA").assign(key=1), on="key")
    .drop("key", axis=1)
)

hospitales = panel_parcial.merge(
    hospitales,
    on=["FECHA", "COD_MUNICIPIO", "MUNICIPIO"],
    how="left"
)

hospitales[cols] = hospitales[cols].apply(pd.to_numeric,errors="coerce").fillna(0)

hospitales.head()

,MUNICIPIO,COD_MUNICIPIO,FECHA,HOSPITALES,CAMAS_HOSPITALARIAS
0,42001 ABEJAR,42001,2000,0.0,0.0
1,42001 ABEJAR,42001,2002,0.0,0.0
2,42001 ABEJAR,42001,2003,0.0,0.0
3,42001 ABEJAR,42001,2004,0.0,0.0
4,42001 ABEJAR,42001,2005,0.0,0.0


FICHERO 9: RESTAURANTES

In [51]:
import pandas as pd

ruta = r"C:\Users\elena\Desktop\universidad\Máster Big Data\Trabajo Fin de Máster\Datos\Restaurantes.csv"

restaurantes = pd.read_csv(ruta,sep=',', encoding='latin1',skiprows=5)

restaurantes.drop(columns=restaurantes.columns[-1], inplace=True)

restaurantes.head()

,FECHA,MUNICIPIO,Sum
0,2000,05002 ADRADA (LA),9
1,NaN,05012 AMAVIDA,1
2,NaN,05013 ARENAL (EL),4
3,NaN,05014 ARENAS DE SAN PEDRO,21
4,NaN,05016 AREVALO,13


In [52]:
restaurantes.rename(
    columns={
        restaurantes.columns[2]: "RESTAURANTES"
    },
    inplace=True
)
restaurantes.head()

,FECHA,MUNICIPIO,RESTAURANTES
0,2000,05002 ADRADA (LA),9
1,NaN,05012 AMAVIDA,1
2,NaN,05013 ARENAL (EL),4
3,NaN,05014 ARENAS DE SAN PEDRO,21
4,NaN,05016 AREVALO,13


In [53]:
cols = [
    "RESTAURANTES",
]

restaurantes[cols] = (
    restaurantes[cols]
    .astype(str)
    .apply(lambda x: x.str.strip())
    .replace(".", 0)
    .apply(pd.to_numeric)
)

#Arreglamos la fecha para que aparezca en todas las filas 

restaurantes["FECHA"] = restaurantes["FECHA"].ffill()

# Nos quedamos con los municipios de Soria, que son los que su codigo postal empieza por 42

restaurantes["COD_MUNICIPIO"] = (
    restaurantes["MUNICIPIO"]
    .astype(str)
    .str.strip()
    .str[:5]
)
restaurantes = restaurantes[
    restaurantes["COD_MUNICIPIO"].str.startswith("42")
]


In [54]:
#los municipios que no aparecen aqui pero cuyo año sí aparece, es porque no tienen ningun restaurante
#para evitar en el modelo confundir un valor NA con lo que deberia ser un cero, vamos a cruzar con la maestra
#de municipios y asignamos los ceros correspondientes

#lo primero que tenemos en cuenta para cruzar son las fechas de alojamientos
anios_existentes = restaurantes["FECHA"].drop_duplicates()


panel_parcial = (
    municipios
    .assign(key=1)
    .merge(anios_existentes.to_frame(name="FECHA").assign(key=1), on="key")
    .drop("key", axis=1)
)

restaurantes = panel_parcial.merge(
    restaurantes,
    on=["FECHA", "COD_MUNICIPIO", "MUNICIPIO"],
    how="left"
)

restaurantes[cols] = restaurantes[cols].apply(pd.to_numeric,errors="coerce").fillna(0)

restaurantes.head()

,MUNICIPIO,COD_MUNICIPIO,FECHA,RESTAURANTES
0,42001 ABEJAR,42001,2000,7.0
1,42001 ABEJAR,42001,2001,7.0
2,42001 ABEJAR,42001,2002,6.0
3,42001 ABEJAR,42001,2003,6.0
4,42001 ABEJAR,42001,2004,6.0


FICHERO 10: PARO REGISTRADO

In [56]:
import pandas as pd

ruta = r"C:\Users\elena\Desktop\universidad\Máster Big Data\Trabajo Fin de Máster\Datos\Paro registrado.csv"

paro = pd.read_csv(ruta,sep=',', encoding='latin1',skiprows=5)

paro.drop(columns=paro.columns[-1], inplace=True)

paro.head()

,FECHA,MUNICIPIO,Sum,Sum.1,Sum.2,Sum.3
0,2000,05001 ADANERO,.,5,1,3
1,NaN,05002 ADRADA (LA),12,28,9,31
2,NaN,05005 ALBORNOS,1,7,.,3
3,NaN,05007 ALDEANUEVA DE SANTA CRUZ,.,3,.,1
4,NaN,05008 ALDEASECA,.,13,.,6


In [57]:
paro.rename(
    columns={
        paro.columns[2]: "PARO_MUJERES_MENOR_25",
        paro.columns[3]: "PARO_MUJERES_MAYOR_25",
        paro.columns[4]: "PARO_VARONES_MENOR_25",
        paro.columns[5]: "PARO_VARONES_MAYOR_25",
    },
    inplace=True
)
paro.head()

,FECHA,MUNICIPIO,PARO_MUJERES_MENOR_25,PARO_MUJERES_MAYOR_25,PARO_VARONES_MENOR_25,PARO_VARONES_MAYOR_25
0,2000,05001 ADANERO,.,5,1,3
1,NaN,05002 ADRADA (LA),12,28,9,31
2,NaN,05005 ALBORNOS,1,7,.,3
3,NaN,05007 ALDEANUEVA DE SANTA CRUZ,.,3,.,1
4,NaN,05008 ALDEASECA,.,13,.,6


In [58]:
cols = [
    "PARO_MUJERES_MENOR_25",
    "PARO_MUJERES_MAYOR_25",
    "PARO_VARONES_MENOR_25",
    "PARO_VARONES_MAYOR_25"
]

paro[cols] = (
    paro[cols]
    .astype(str)
    .apply(lambda x: x.str.strip())
    .replace(".", 0)
    .apply(pd.to_numeric)
)

#Arreglamos la fecha para que aparezca en todas las filas 

paro["FECHA"] = paro["FECHA"].ffill()

# Nos quedamos con los municipios de Soria, que son los que su codigo postal empieza por 42

paro["COD_MUNICIPIO"] = (
    paro["MUNICIPIO"]
    .astype(str)
    .str.strip()
    .str[:5]
)
paro = paro[
    paro["COD_MUNICIPIO"].str.startswith("42")
]

In [59]:
#los municipios que no aparecen aqui pero cuyo año sí aparece, es porque no tienen ninguna demanda de paro
#para evitar en el modelo confundir un valor NA con lo que deberia ser un cero, vamos a cruzar con la maestra
#de municipios y asignamos los ceros correspondientes

#lo primero que tenemos en cuenta para cruzar son las fechas de alojamientos
anios_existentes = paro["FECHA"].drop_duplicates()


panel_parcial = (
    municipios
    .assign(key=1)
    .merge(anios_existentes.to_frame(name="FECHA").assign(key=1), on="key")
    .drop("key", axis=1)
)

paro = panel_parcial.merge(
    paro,
    on=["FECHA", "COD_MUNICIPIO", "MUNICIPIO"],
    how="left"
)

paro[cols] = paro[cols].apply(pd.to_numeric,errors="coerce").fillna(0)

paro.head()

,MUNICIPIO,COD_MUNICIPIO,FECHA,PARO_MUJERES_MENOR_25,PARO_MUJERES_MAYOR_25,PARO_VARONES_MENOR_25,PARO_VARONES_MAYOR_25
0,42001 ABEJAR,42001,2000,0.0,10.0,0.0,3.0
1,42001 ABEJAR,42001,2001,0.0,5.0,0.0,3.0
2,42001 ABEJAR,42001,2002,1.0,5.0,1.0,3.0
3,42001 ABEJAR,42001,2003,0.0,4.0,0.0,4.0
4,42001 ABEJAR,42001,2004,0.0,2.0,0.0,3.0


FICHERO 11: VARIACIONES RESIDENCIALES

In [61]:
import pandas as pd

ruta = r"C:\Users\elena\Desktop\universidad\Máster Big Data\Trabajo Fin de Máster\Datos\Variaciones residenciales.csv"

variaciones_residenciales = pd.read_csv(ruta,sep=',', encoding='latin1',skiprows=5, dtype={0: "string"})

variaciones_residenciales.drop(columns=variaciones_residenciales.columns[-1], inplace=True)

variaciones_residenciales.head()

,FECHA,MUNICIPIO,Sum,Sum.1,Sum.2,Sum.3,Sum.4,Sum.5,Sum.6
0,1990,05001 ADANERO,5,.,3,.,13,7,.
1,<NA>,05002 ADRADA (LA),5,12,.,1,60,53,.
2,<NA>,05005 ALBORNOS,5,.,2,.,2,.,.
3,<NA>,05007 ALDEANUEVA DE SANTA CRUZ,5,2,.,.,5,1,.
4,<NA>,05008 ALDEASECA,2,4,.,.,11,.,.


In [62]:
variaciones_residenciales.rename(
    columns={
        variaciones_residenciales.columns[2]: "EMIGRACIONES CON DESTINO A LA MISMA PROVINCIA",
        variaciones_residenciales.columns[3]: "INMIGRACIONES PROCEDENTES DE LA MISMA PROVINCIA",
        variaciones_residenciales.columns[4]: "EMIGRACIONES CON DESTINO A OTRA PROVINCIA DE CASTILLA Y LEÓN",
        variaciones_residenciales.columns[5]: "INMIGRACIONES PROCEDENTES DE OTRA PROVINCIA DE CASTILLA Y LEÓN",
        variaciones_residenciales.columns[6]: "EMIGRACIONES CON DESTINO A OTRAS COMUNIDADES AUTÓNOMAS",
        variaciones_residenciales.columns[7]: "INMIGRACIONES PROCEDENTES DE OTRAS COMUNIDADES AUTÓNOMAS",
        variaciones_residenciales.columns[8]: "EMIGRACIONES CON DESTINO A OTROS PAÍSES"
    },
    inplace=True
)
variaciones_residenciales.head()

,FECHA,MUNICIPIO,EMIGRACIONES CON DESTINO A LA MISMA PROVINCIA,INMIGRACIONES PROCEDENTES DE LA MISMA PROVINCIA,EMIGRACIONES CON DESTINO A OTRA PROVINCIA DE CASTILLA Y LEÓN,INMIGRACIONES PROCEDENTES DE OTRA PROVINCIA DE CASTILLA Y LEÓN,EMIGRACIONES CON DESTINO A OTRAS COMUNIDADES AUTÓNOMAS,INMIGRACIONES PROCEDENTES DE OTRAS COMUNIDADES AUTÓNOMAS,EMIGRACIONES CON DESTINO A OTROS PAÍSES
0,1990,05001 ADANERO,5,.,3,.,13,7,.
1,<NA>,05002 ADRADA (LA),5,12,.,1,60,53,.
2,<NA>,05005 ALBORNOS,5,.,2,.,2,.,.
3,<NA>,05007 ALDEANUEVA DE SANTA CRUZ,5,2,.,.,5,1,.
4,<NA>,05008 ALDEASECA,2,4,.,.,11,.,.


In [63]:
cols = [
    "EMIGRACIONES CON DESTINO A LA MISMA PROVINCIA",
    "INMIGRACIONES PROCEDENTES DE LA MISMA PROVINCIA",
    "EMIGRACIONES CON DESTINO A OTRA PROVINCIA DE CASTILLA Y LEÓN",
    "INMIGRACIONES PROCEDENTES DE OTRA PROVINCIA DE CASTILLA Y LEÓN",
    "EMIGRACIONES CON DESTINO A OTRAS COMUNIDADES AUTÓNOMAS",
    "INMIGRACIONES PROCEDENTES DE OTRAS COMUNIDADES AUTÓNOMAS",
    "EMIGRACIONES CON DESTINO A OTROS PAÍSES"
]

variaciones_residenciales[cols] = (
    variaciones_residenciales[cols]
    .astype(str)
    .apply(lambda x: x.str.strip())
    .replace(".", 0)
    .apply(pd.to_numeric)
)

#Arreglamos la fecha para que aparezca en todas las filas 

variaciones_residenciales["FECHA"] = variaciones_residenciales["FECHA"].ffill()

variaciones_residenciales["FECHA"] = variaciones_residenciales["FECHA"].astype(str).str.strip().str[:4]

# Nos quedamos con los municipios de Soria, que son los que su codigo postal empieza por 42

variaciones_residenciales["COD_MUNICIPIO"] = (
    variaciones_residenciales["MUNICIPIO"]
    .astype(str)
    .str.strip()
    .str[:5]
)
variaciones_residenciales = variaciones_residenciales[
    variaciones_residenciales["COD_MUNICIPIO"].str.startswith("42")
]

In [64]:
#los municipios que no aparecen aqui pero cuyo año sí aparece, es porque no tienen ninguna var. residencial
#para evitar en el modelo confundir un valor NA con lo que deberia ser un cero, vamos a cruzar con la maestra
#de municipios y asignamos los ceros correspondientes

#lo primero que tenemos en cuenta para cruzar son las fechas de alojamientos
anios_existentes = variaciones_residenciales["FECHA"].drop_duplicates()


panel_parcial = (
    municipios
    .assign(key=1)
    .merge(anios_existentes.to_frame(name="FECHA").assign(key=1), on="key")
    .drop("key", axis=1)
)

variaciones_residenciales = panel_parcial.merge(
    variaciones_residenciales,
    on=["FECHA", "COD_MUNICIPIO", "MUNICIPIO"],
    how="left"
)

variaciones_residenciales[cols] = variaciones_residenciales[cols].apply(pd.to_numeric,errors="coerce").fillna(0)

variaciones_residenciales.head()

,MUNICIPIO,COD_MUNICIPIO,FECHA,EMIGRACIONES CON DESTINO A LA MISMA PROVINCIA,INMIGRACIONES PROCEDENTES DE LA MISMA PROVINCIA,EMIGRACIONES CON DESTINO A OTRA PROVINCIA DE CASTILLA Y LEÓN,INMIGRACIONES PROCEDENTES DE OTRA PROVINCIA DE CASTILLA Y LEÓN,EMIGRACIONES CON DESTINO A OTRAS COMUNIDADES AUTÓNOMAS,INMIGRACIONES PROCEDENTES DE OTRAS COMUNIDADES AUTÓNOMAS,EMIGRACIONES CON DESTINO A OTROS PAÍSES
0,42001 ABEJAR,42001,1990,8.0,15.0,0.0,0.0,4.0,8.0,0.0
1,42001 ABEJAR,42001,1991,4.0,0.0,0.0,1.0,5.0,0.0,0.0
2,42001 ABEJAR,42001,1992,6.0,5.0,0.0,0.0,6.0,6.0,0.0
3,42001 ABEJAR,42001,1993,6.0,15.0,0.0,1.0,8.0,5.0,0.0
4,42001 ABEJAR,42001,1994,2.0,10.0,0.0,0.0,2.0,5.0,0.0


FICHERO 12: POBLACION POR SEXO

In [66]:
import pandas as pd

ruta = r"C:\Users\elena\Desktop\universidad\Máster Big Data\Trabajo Fin de Máster\Datos\Población.csv"

poblacion_sexo = pd.read_csv(ruta,sep=',', encoding='latin1',skiprows=5)

poblacion_sexo.drop(columns=poblacion_sexo.columns[-1], inplace=True)

poblacion_sexo.head()

,FECHA,MUNICIPIO,Sum,Sum.1,Sum.2
0,1986,05001 ADANERO,419,191,228
1,NaN,05002 ADRADA (LA),1832,884,948
2,NaN,05005 ALBORNOS,326,151,175
3,NaN,05007 ALDEANUEVA DE SANTA CRUZ,237,110,127
4,NaN,05008 ALDEASECA,446,223,223


In [67]:
poblacion_sexo.rename(
    columns={
        poblacion_sexo.columns[2]: "POBLACION_TOTAL",
        poblacion_sexo.columns[3]: "POBLACION_MUJERES",
        poblacion_sexo.columns[4]: "POBLACION_HOMBRES",
    },
    inplace=True
)
poblacion_sexo.head()

,FECHA,MUNICIPIO,POBLACION_TOTAL,POBLACION_MUJERES,POBLACION_HOMBRES
0,1986,05001 ADANERO,419,191,228
1,NaN,05002 ADRADA (LA),1832,884,948
2,NaN,05005 ALBORNOS,326,151,175
3,NaN,05007 ALDEANUEVA DE SANTA CRUZ,237,110,127
4,NaN,05008 ALDEASECA,446,223,223


In [68]:
cols = [
    "POBLACION_TOTAL",
    "POBLACION_HOMBRES",
    "POBLACION_MUJERES"
]

poblacion_sexo[cols] = (
    poblacion_sexo[cols]
    .astype(str)
    .apply(lambda x: x.str.strip())
    .replace(".", 0)
    .apply(pd.to_numeric)
)

#Arreglamos la fecha para que aparezca en todas las filas 

poblacion_sexo["FECHA"] = poblacion_sexo["FECHA"].ffill()

poblacion_sexo["FECHA"] = poblacion_sexo["FECHA"].astype(str).str.strip().str[:4]

# Nos quedamos con los municipios de Soria, que son los que su codigo postal empieza por 42

poblacion_sexo["COD_MUNICIPIO"] = (
    poblacion_sexo["MUNICIPIO"]
    .astype(str)
    .str.strip()
    .str[:5]
)
poblacion_sexo = poblacion_sexo[
    poblacion_sexo["COD_MUNICIPIO"].str.startswith("42")
]

poblacion_sexo.head()

#todos los municipios tienen poblacion por lo tatno no hay que cruzar con la maestra de municipios

,FECHA,MUNICIPIO,POBLACION_TOTAL,POBLACION_MUJERES,POBLACION_HOMBRES,COD_MUNICIPIO
1591,1986,42001 ABEJAR,376,182,194,42001
1592,1986,42003 ADRADAS,140,67,73,42003
1593,1986,42004 AGREDA,3740,1929,1811,42004
1594,1986,42006 ALCONABA,163,71,92,42006
1595,1986,42007 ALCUBILLA DE AVELLANEDA,308,147,161,42007


FICHERO 13: POBLACION POR EDAD Y NACIONALIDAD

In [70]:
import pandas as pd

ruta = r"C:\Users\elena\Desktop\universidad\Máster Big Data\Trabajo Fin de Máster\Datos\INE - Cifras de población.xlsx"

poblacion_raw = pd.read_excel(
    ruta,
    sheet_name=0,
    header=None
)

poblacion_raw.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,,Total,Total,Total,Total,Española,Española,Española,Española,Extranjera,Extranjera,Extranjera,Extranjera
1,,Todas las edades,Menos de 16 años,De 16 a 64 años,65 y más años,Todas las edades,Menos de 16 años,De 16 a 64 años,65 y más años,Todas las edades,Menos de 16 años,De 16 a 64 años,65 y más años
2,42001 Abejar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1 de enero de 2025,285,24,167,94,231,17,120,94,54,7,47,0
4,1 de enero de 2024,383,46,242,95,237,16,127,94,146,30,115,1


In [71]:
fila_nacionalidad = poblacion_raw.iloc[0]
fila_edad = poblacion_raw.iloc[1]

columnas = ["AUX"] + [
    f"{fila_nacionalidad[i]}_{fila_edad[i]}"
    for i in range(1, len(fila_nacionalidad))
]

columnas = (
    pd.Series(columnas)
    .str.replace(" ", "_")
    .str.replace("á", "a")
    .str.replace("é", "e")
    .str.replace("í", "i")
    .str.replace("ó", "o")
    .str.replace("ú", "u")
    .tolist()
)

In [72]:
datos = poblacion_raw.iloc[2:].reset_index(drop=True)
datos.columns = columnas

In [73]:
datos["MUNICIPIO"] = datos["AUX"].where(
    datos["AUX"]
    .astype(str)
    .str.strip()
    .str.match(r"^42\d{3}\s")
)

datos["FECHA"] = datos["AUX"].where(
    datos["AUX"].astype(str).str.contains("enero")
)

datos[["MUNICIPIO", "FECHA"]].head(20)

,MUNICIPIO,FECHA
0,42001 Abejar,NaN
1,NaN,1 de enero de 2025
2,NaN,1 de enero de 2024
3,NaN,1 de enero de 2023
4,NaN,1 de enero de 2022
5,NaN,1 de enero de 2021
6,NaN,1 de enero de 2020
7,NaN,1 de enero de 2019
8,NaN,1 de enero de 2018
9,NaN,1 de enero de 2017


In [74]:
datos["MUNICIPIO"] = datos["MUNICIPIO"].ffill()

datos.drop(columns=["AUX"], inplace=True)

poblacion_edad = datos[datos["FECHA"].notna()].copy()

poblacion_edad["FECHA"] = (
    poblacion_edad["FECHA"]
    .astype(str)
    .str[-4:]
)

poblacion_edad["COD_MUNICIPIO"] = (
    poblacion_edad["MUNICIPIO"]
    .astype(str)
    .str.strip()
    .str[:5]
)

poblacion_edad.head(20)

#todos los municipios tienen poblacion por lo tanto no hace falta cruzar con la maestra de municipios

,Total_Todas_las_edades,Total_Menos_de_16_años,Total_De_16_a_64_años,Total_65_y_mas_años,Española_Todas_las_edades,Española_Menos_de_16_años,Española_De_16_a_64_años,Española_65_y_mas_años,Extranjera_Todas_las_edades,Extranjera_Menos_de_16_años,Extranjera_De_16_a_64_años,Extranjera_65_y_mas_años,MUNICIPIO,FECHA,COD_MUNICIPIO
1,285,24,167,94,231,17,120,94,54,7,47,0,42001 Abejar,2025,42001
2,383,46,242,95,237,16,127,94,146,30,115,1,42001 Abejar,2024,42001
3,377,53,222,102,251,19,132,100,126,34,90,2,42001 Abejar,2023,42001
4,273,22,151,100,243,17,126,100,30,5,25,0,42001 Abejar,2022,42001
5,273,18,155,100,244,14,130,100,29,4,25,0,42001 Abejar,2021,42001
6,286,22,164,100,262,18,144,100,24,4,20,0,42001 Abejar,2020,42001
7,297,23,171,103,268,18,147,103,29,5,24,0,42001 Abejar,2019,42001
8,302,15,181,106,272,12,154,106,30,3,27,0,42001 Abejar,2018,42001
9,318,19,187,112,292,15,165,112,26,4,22,0,42001 Abejar,2017,42001
10,327,21,193,113,302,16,173,113,25,5,20,0,42001 Abejar,2016,42001


FICHERO 14: ALUMNOS

In [76]:
import pandas as pd

ruta = r"C:\Users\elena\Desktop\universidad\Máster Big Data\Trabajo Fin de Máster\Datos\alumnos.xlsx"

alumnos_raw = pd.read_excel(
    ruta,
    sheet_name=0,
    header=None
)

alumnos_raw.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,Curso académico,2005,2006,2007,2008,2009,2010,2011,2012,2013,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
1,Municipio,Sum,Sum,Sum,Sum,Sum,Sum,Sum,Sum,Sum,...,Sum,Sum,Sum,Sum,Sum,Sum,Sum,Sum,Sum,Sum
2,42001 ABEJAR,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,42003 ADRADAS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,42004 AGREDA,461,448,482,448,523,497,515,511,487,...,472,493,460,463,479,450,481,510,518,537


In [77]:
fechas = alumnos_raw.iloc[0, 1:].astype(str)

alumnos_long = alumnos_raw.iloc[2:].reset_index(drop=True)

alumnos_long.columns = ["MUNICIPIO"] + fechas.tolist()

alumnos = alumnos_long.melt(
    id_vars="MUNICIPIO",
    var_name="FECHA",
    value_name="ALUMNOS"
)

alumnos["COD_MUNICIPIO"] = (
    alumnos["MUNICIPIO"]
    .astype(str)
    .str.strip()
    .str[:5]
)
alumnos.head()
#el fichero ya tiene todos los municipios, no hay que cruzar con la maestra

,MUNICIPIO,FECHA,ALUMNOS,COD_MUNICIPIO
0,42001 ABEJAR,2005,0,42001
1,42003 ADRADAS,2005,0,42003
2,42004 AGREDA,2005,461,42004
3,42006 ALCONABA,2005,0,42006
4,42007 ALCUBILLA DE AVELLANEDA,2005,0,42007


FICHERO 15: COLEGIOS

In [79]:
import pandas as pd

ruta = r"C:\Users\elena\Desktop\universidad\Máster Big Data\Trabajo Fin de Máster\Datos\centros educativos.xlsx"

escuelas_raw = pd.read_excel(
    ruta,
    sheet_name=0,
    header=None
)

escuelas_raw.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,NaN,2005,2006,2007,2008,2009,2010,2011,2012,2013,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
1,42001 ABEJAR,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,42003 ADRADAS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,42004 AGREDA,2,2,2,2,2,2,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,42006 ALCONABA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
fechas = escuelas_raw.iloc[0, 1:].astype(str)

escuelas_long = escuelas_raw.iloc[2:].reset_index(drop=True)

escuelas_long.columns = ["MUNICIPIO"] + fechas.tolist()

escuelas = escuelas_long.melt(
    id_vars="MUNICIPIO",
    var_name="FECHA",
    value_name="ESCUELAS"
)

escuelas["COD_MUNICIPIO"] = (
    escuelas["MUNICIPIO"]
    .astype(str)
    .str.strip()
    .str[:5]
)
escuelas.head()

#el fichero ya tiene todos los municipios, no hay que cruzar con la maestra

,MUNICIPIO,FECHA,ESCUELAS,COD_MUNICIPIO
0,42003 ADRADAS,2005,0,42003
1,42004 AGREDA,2005,2,42004
2,42006 ALCONABA,2005,0,42006
3,42007 ALCUBILLA DE AVELLANEDA,2005,0,42007
4,42008 ALCUBILLA DE LAS PEÑAS,2005,0,42008


FICHERO 16: CAMPAMENTOS DE TURISMO

In [82]:
import pandas as pd

ruta = r"C:\Users\elena\Desktop\universidad\Máster Big Data\Trabajo Fin de Máster\Datos\Campamentos de turismo.csv"

campamentos = pd.read_csv(ruta,sep=',', encoding='latin1',skiprows=5)

campamentos.drop(columns=campamentos.columns[-1], inplace=True)

campamentos.head()

,FECHA,MUNICIPIO,Sum
0,2000,05019 AVILA,1
1,NaN,05022 BARRACO (EL),1
2,NaN,05041 BURGOHONDO,1
3,NaN,05054 CASAVIEJA,1
4,NaN,05089 GUISANDO,1


In [83]:
campamentos.rename(
    columns={
        campamentos.columns[2]: "CAMPAMENTOS_TURISMO",
    },
    inplace=True
)
campamentos.head()

,FECHA,MUNICIPIO,CAMPAMENTOS_TURISMO
0,2000,05019 AVILA,1
1,NaN,05022 BARRACO (EL),1
2,NaN,05041 BURGOHONDO,1
3,NaN,05054 CASAVIEJA,1
4,NaN,05089 GUISANDO,1


In [84]:
cols = [
    "CAMPAMENTOS_TURISMO"
]

campamentos[cols] = (
    campamentos[cols]
    .astype(str)
    .apply(lambda x: x.str.strip())
    .replace(".", 0)
    .apply(pd.to_numeric)
)

#Arreglamos la fecha para que aparezca en todas las filas 

campamentos["FECHA"] = campamentos["FECHA"].ffill()

# Nos quedamos con los municipios de Soria, que son los que su codigo postal empieza por 42

campamentos["COD_MUNICIPIO"] = (
    campamentos["MUNICIPIO"]
    .astype(str)
    .str.strip()
    .str[:5]
)
campamentos = campamentos[
    campamentos["COD_MUNICIPIO"].str.startswith("42")
]

In [85]:
#los municipios que no aparecen aqui pero cuyo año sí aparece, es porque no tienen ningun campamento
#para evitar en el modelo confundir un valor NA con lo que deberia ser un cero, vamos a cruzar con la maestra
#de municipios y asignamos los ceros correspondientes

#lo primero que tenemos en cuenta para cruzar son las fechas de alojamientos
anios_existentes = campamentos["FECHA"].drop_duplicates()


panel_parcial = (
    municipios
    .assign(key=1)
    .merge(anios_existentes.to_frame(name="FECHA").assign(key=1), on="key")
    .drop("key", axis=1)
)

campamentos = panel_parcial.merge(
    campamentos,
    on=["FECHA", "COD_MUNICIPIO", "MUNICIPIO"],
    how="left"
)

campamentos[cols] = campamentos[cols].apply(pd.to_numeric,errors="coerce").fillna(0)

campamentos.head()

,MUNICIPIO,COD_MUNICIPIO,FECHA,CAMPAMENTOS_TURISMO
0,42001 ABEJAR,42001,2000,2.0
1,42001 ABEJAR,42001,2001,2.0
2,42001 ABEJAR,42001,2002,2.0
3,42001 ABEJAR,42001,2003,2.0
4,42001 ABEJAR,42001,2004,2.0


JUNTAMOS TODOS LOS FICHEROS

NOMBRES DE FICHEROS Y VARIABLES: 


In [88]:
#voy a eliminar la variable municipio de todos los dataframes menos del primero, para que no se me duplique la variable y solo me traiga las columnas nuevas
lista_dfs = [
    poblacion_sexo,
    poblacion_edad.drop(columns=["MUNICIPIO"], errors="ignore"),
    variaciones_residenciales.drop(columns=["MUNICIPIO"], errors="ignore"),
    alojamientos.drop(columns=["MUNICIPIO"], errors="ignore"),
    campamentos.drop(columns=["MUNICIPIO"], errors="ignore"),
    restaurantes.drop(columns=["MUNICIPIO"], errors="ignore"),
    consultorios.drop(columns=["MUNICIPIO"], errors="ignore"),
    farmacias.drop(columns=["MUNICIPIO"], errors="ignore"),
    hospitales.drop(columns=["MUNICIPIO"], errors="ignore"),
    empleo.drop(columns=["MUNICIPIO"], errors="ignore"),
    paro.drop(columns=["MUNICIPIO"], errors="ignore"),
    empresas.drop(columns=["MUNICIPIO"], errors="ignore"),
    familias_agricultoras.drop(columns=["MUNICIPIO"], errors="ignore"),
    alumnos.drop(columns=["MUNICIPIO"], errors="ignore"),
    escuelas.drop(columns=["MUNICIPIO"], errors="ignore")
]

In [89]:
from functools import reduce

#lista_dfs = [poblacion_sexo, poblacion_edad, variaciones_residenciales, alojamientos, campamentos,restaurantes, consultorios, farmacias, hospitlaes, empleo, paro, empresas, familias_agricultoras, 
 #            alumnos,escuelas] #distancia a la capital no es por fecha, por lo tanto la tendremos que añadir despues
for i, df in enumerate(lista_dfs):
    lista_dfs[i] = df.assign(
        FECHA=df["FECHA"].astype(str).str[:4]
    )
    
df_final = reduce(
    lambda left, right: pd.merge(
        left, right,
        on=["FECHA", "COD_MUNICIPIO"],
        how="left"
    ),
    lista_dfs
)

In [90]:
df_final.head()

,FECHA,MUNICIPIO,POBLACION_TOTAL,POBLACION_MUJERES,POBLACION_HOMBRES,COD_MUNICIPIO,Total_Todas_las_edades,Total_Menos_de_16_años,Total_De_16_a_64_años,Total_65_y_mas_años,...,DEMANDAS_VARONES_MENOR_25,DEMANDAS_VARONES_MAYOR_25,PARO_MUJERES_MENOR_25,PARO_MUJERES_MAYOR_25,PARO_VARONES_MENOR_25,PARO_VARONES_MAYOR_25,EMPRESAS,FAMILIARES,ALUMNOS,ESCUELAS
0,1986,42001 ABEJAR,376,182,194,42001,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1986,42003 ADRADAS,140,67,73,42003,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1986,42004 AGREDA,3740,1929,1811,42004,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1986,42006 ALCONABA,163,71,92,42006,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1986,42007 ALCUBILLA DE AVELLANEDA,308,147,161,42007,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
df_final["FECHA"].unique()

array(['1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993',
       '1994', '1995', '1996', '1998', '1999', '2000', '2001', '2002',
       '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019', '2020', '2021', '2022', '2023', '2024', 'TOTA'],
      dtype=object)

In [92]:
#eliminamos la fecha incorrecta
df_final = df_final[
    df_final["FECHA"].notna() &
    (df_final["FECHA"].astype(str) != "NaT") &
    (df_final["FECHA"].astype(str) != "TOTA")
].copy()

df_final["FECHA"] = df_final["FECHA"].astype(int)

In [93]:
df_final.shape

(6954, 45)

In [94]:
datasets = {
    "poblacion_sexo": poblacion_sexo,
    "poblacion_edad": poblacion_edad,
    "variaciones_residenciales": variaciones_residenciales,
    "alojamientos": alojamientos,
    "campamentos": campamentos,
    "restaurantes": restaurantes,
    "consultorios": consultorios,
    "farmacias": farmacias,
    "hospitales": hospitales,
    "empleo": empleo,
    "paro": paro,
    "empresas": empresas,
    "familias_agricultoras": familias_agricultoras,
    "alumnos": alumnos,
    "escuelas": escuelas
}

for df in datasets.values():
    df["FECHA"] = pd.to_datetime(df["FECHA"], errors="coerce")

resumen_fechas = []

for nombre, df in datasets.items():
    resumen_fechas.append({
        "dataset": nombre,
        "fecha_min": df["FECHA"].min(),
        "fecha_max": df["FECHA"].max(),
        "n_filas": len(df)
    })

df_resumen = pd.DataFrame(resumen_fechas)
df_resumen

,dataset,fecha_min,fecha_max,n_filas
0,poblacion_sexo,1986-01-01,2024-01-01,7137
1,poblacion_edad,2003-01-01,2025-01-01,4200
2,variaciones_residenciales,1990-01-01,2021-01-01,6039
3,alojamientos,2000-01-01,2024-01-01,4758
4,campamentos,2000-01-01,2024-01-01,4758
5,restaurantes,2000-01-01,2024-01-01,4758
6,consultorios,2000-01-01,2024-01-01,3660
7,farmacias,2000-01-01,2024-01-01,2928
8,hospitales,2000-01-01,2024-01-01,4575
9,empleo,2000-01-01,2024-01-01,4758


Filtramos el periodo 2005-2024, y decidimos excluir del modelo el fichero de familias agricultoras

In [96]:
#filtramos fechas y borramos columnas
df_final = (
    df_final
    .loc[df_final["FECHA"].between(2005, 2024)]
    .drop(columns=["FAMILIARES"])
    .copy()
)

In [97]:
df_final.head(300)

,FECHA,MUNICIPIO,POBLACION_TOTAL,POBLACION_MUJERES,POBLACION_HOMBRES,COD_MUNICIPIO,Total_Todas_las_edades,Total_Menos_de_16_años,Total_De_16_a_64_años,Total_65_y_mas_años,...,DEMANDAS_MUJERES_MAYOR_25,DEMANDAS_VARONES_MENOR_25,DEMANDAS_VARONES_MAYOR_25,PARO_MUJERES_MENOR_25,PARO_MUJERES_MAYOR_25,PARO_VARONES_MENOR_25,PARO_VARONES_MAYOR_25,EMPRESAS,ALUMNOS,ESCUELAS
3294,2005,42001 ABEJAR,392,187,205,42001,392,33,239,120,...,4.0,0.0,5.0,1.0,2.0,0.0,4.0,NaN,0,NaN
3295,2005,42003 ADRADAS,82,37,45,42003,82,4,42,36,...,3.0,0.0,0.0,0.0,3.0,0.0,0.0,NaN,0,0.0
3296,2005,42004 AGREDA,3216,1599,1617,42004,3216,416,1944,856,...,62.0,5.0,30.0,3.0,55.0,3.0,24.0,NaN,461,2.0
3297,2005,42006 ALCONABA,171,69,102,42006,171,14,113,44,...,2.0,0.0,5.0,0.0,2.0,0.0,2.0,NaN,0,0.0
3298,2005,42007 ALCUBILLA DE AVELLANEDA,183,93,90,42007,183,5,83,95,...,4.0,0.0,1.0,0.0,4.0,0.0,1.0,NaN,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3589,2006,42132 NOVIERCAS,206,98,108,42132,206,9,106,91,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,NaN,0,0.0
3590,2006,42134 OLVEGA,3484,1676,1808,42134,3484,509,2340,635,...,84.0,4.0,21.0,1.0,73.0,3.0,14.0,NaN,599,4.0
3591,2006,42135 ONCALA,104,41,63,42135,104,8,54,42,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,NaN,0,0.0
3592,2006,42139 PINILLA DEL CAMPO,21,9,12,42139,21,0,12,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,0.0


Vamos a realizar agregraciones de variables según el concepto que representan: 

"CAMPAMENTOS_TURISMO" + "ALOJAMIENTOS_HOTELEROS" + "ALOJAMIENTOS_TURISMO_RURAL" + "RESTAURANTES" se agrupará en una variable llamada servicios turisticos.

"CONSULTORIOS" + "FARMACIAS" + "HOSPITAL Se agrupará en una variable llamada servicios sanitariosESSe elimina la columna rop  "CAMAS_HOSPITALA, dado que a nivel municipal, ningún municipio a excepción de la capital tiene camas hospitalarias.AS"



"DEMANDAS_MUJERES_MENOR_25" + "DEMANDAS_MUJERES_MAYOR_25" + "DEMANDAS_VARONES_MENOR_25" + "DEMANDAS_VARONES_Mse crea una variable agrupada más general que no distinga por edad ni sexo. 
A5" 

    "PARO_MUJERES_MENOR_25" + "PARO_MUJERES_MAYOR_25" + "PARO_VARONES_MENOR_25" + "PARO_VARONE se crea una variable agrupada más general que no distinga por edad ni sexo. 

prop inmigrantes = Extranjera_Todas_las_edades / Total_Todas_las_edades

índice envejecimiento = Total_65_y_mas_años / Total_Todas_las_edad

El numero de habitantes menores de 16 esta muy relacionado con la variable alumnos, por lo tanto, vamos a eliminar la primera y nos quedamos con el numero de alumnos para nuestro analisis.esSe eliminan las siguientes columnas al haberse creado los dos indicadores anteriores (y tener la variable alumnos): rop Total_Todas_las_edades Total_Menos_de_16_años	Total_De_16_a_64_años	Total_65_y_mas_años	Española_Todas_las_edades	Española_Menos_de_16_años	Española_De_16_a_64_años	Española_65_y_mas_años	Extranjera_Todas_las_edades	Extranjera_Menos_de_16_años	Extranjera_De_16_a_64_años	Extranjera_65_y_mas_
añosS_MAYOR_25"

In [99]:
df_final = (
    df_final
    # Servicios turísticos
    .assign(
        SERVICIOS_TURISTICOS=
            df_final["CAMPAMENTOS_TURISMO"]
            + df_final["ALOJAMIENTOS_HOTELEROS"]
            + df_final["ALOJAMIENTOS_TURISMO_RURAL"]
            + df_final["RESTAURANTES"],

        # Servicios de sanidad
        SERVICIOS_SANIDAD=
            df_final["CONSULTORIOS"]
            + df_final["FARMACIAS"]
            + df_final["HOSPITALES"],

        # Demandas de empleo
        DEMANDAS_EMPLEO=
            df_final["DEMANDAS_MUJERES_MENOR_25"]
            + df_final["DEMANDAS_MUJERES_MAYOR_25"]
            + df_final["DEMANDAS_VARONES_MENOR_25"]
            + df_final["DEMANDAS_VARONES_MAYOR_25"],

        # Paro total
        PARO=
            df_final["PARO_MUJERES_MENOR_25"]
            + df_final["PARO_MUJERES_MAYOR_25"]
            + df_final["PARO_VARONES_MENOR_25"]
            + df_final["PARO_VARONES_MAYOR_25"],
        
        EMIGRACIONES_TOTALES=
        df_final["EMIGRACIONES CON DESTINO A LA MISMA PROVINCIA"]
        + df_final["EMIGRACIONES CON DESTINO A OTRA PROVINCIA DE CASTILLA Y LEÓN"]
        + df_final["EMIGRACIONES CON DESTINO A OTRAS COMUNIDADES AUTÓNOMAS"]
        + df_final["EMIGRACIONES CON DESTINO A OTROS PAÍSES"],

    INMIGRACIONES_TOTALES=
        df_final["INMIGRACIONES PROCEDENTES DE LA MISMA PROVINCIA"]
        + df_final["INMIGRACIONES PROCEDENTES DE OTRA PROVINCIA DE CASTILLA Y LEÓN"]
        + df_final["INMIGRACIONES PROCEDENTES DE OTRAS COMUNIDADES AUTÓNOMAS"]

    )

    # Eliminar columna no deseada
    .drop(columns=["CAMAS_HOSPITALARIAS"])
)


Construimos varios indicadores

In [111]:
df_final = df_final.assign(
    TASA_PARO = df_final["PARO"] / df_final["Total_De_16_a_64_años"] * 100,
    TASA_DEMANDA = df_final["DEMANDAS_EMPLEO"] / df_final["Total_De_16_a_64_años"] * 100,
    RATIO_DEMANDA_PARO = (df_final["DEMANDAS_EMPLEO"] / df_final["PARO"]).replace([np.inf, -np.inf], np.nan),
    PROP_POBLACION_EXTRANJERA = df_final["Extranjera_Todas_las_edades"] / df_final["Total_Todas_las_edades"],
    PROP_ENVEJECIMIENTO = df_final["Total_65_y_mas_años"] / df_final["Total_Todas_las_edades"],  
    #Capacidad del municipio para atraer población “externa”
    INDICE_ATRACCION_EXTERNA= (df_final["INMIGRACIONES PROCEDENTES DE OTRAS COMUNIDADES AUTÓNOMAS"] + df_final["INMIGRACIONES PROCEDENTES DE OTRA PROVINCIA DE CASTILLA Y LEÓN"]) / df_final["INMIGRACIONES_TOTALES"]  
)

df_final["RATIO_DEMANDA_PARO"].isna().mean()

0.3418032786885246

El indicador RATIO_DEMANDA_PARO presenta valores no definidos en aproximadamente el 28 % de las observaciones debido a la existencia de municipios y años con paro registrado igual a cero.

Una vez creados los indicadores borramos variables que no van a ser utilizadas de forma individual

In [114]:
df_final = df_final.drop(columns=[
    "Total_Todas_las_edades",
    "Total_Menos_de_16_años",
    "Total_De_16_a_64_años",
    "Total_65_y_mas_años",
    "Española_Todas_las_edades",
    "Española_Menos_de_16_años",
    "Española_De_16_a_64_años",
    "Española_65_y_mas_años",
    "Extranjera_Todas_las_edades",
    "Extranjera_Menos_de_16_años",
    "Extranjera_De_16_a_64_años",
    "Extranjera_65_y_mas_años",
    "CAMPAMENTOS_TURISMO",
    "ALOJAMIENTOS_HOTELEROS",
    "ALOJAMIENTOS_TURISMO_RURAL",
    "RESTAURANTES",
    "DEMANDAS_MUJERES_MENOR_25", 
    "DEMANDAS_MUJERES_MAYOR_25",
    "DEMANDAS_VARONES_MENOR_25",
    "DEMANDAS_VARONES_MAYOR_25", 
    "PARO_MUJERES_MENOR_25",
    "PARO_MUJERES_MAYOR_25",
    "PARO_VARONES_MENOR_25",
    "PARO_VARONES_MAYOR_25"
])

Mostramos todas las columnas resultantes para ver que esten correctas y volvemos a repasar si queremos borrar alguna más

In [116]:
df_final.head(100)

,FECHA,MUNICIPIO,POBLACION_TOTAL,POBLACION_MUJERES,POBLACION_HOMBRES,COD_MUNICIPIO,EMIGRACIONES CON DESTINO A LA MISMA PROVINCIA,INMIGRACIONES PROCEDENTES DE LA MISMA PROVINCIA,EMIGRACIONES CON DESTINO A OTRA PROVINCIA DE CASTILLA Y LEÓN,INMIGRACIONES PROCEDENTES DE OTRA PROVINCIA DE CASTILLA Y LEÓN,...,DEMANDAS_EMPLEO,PARO,EMIGRACIONES_TOTALES,INMIGRACIONES_TOTALES,TASA_PARO,TASA_DEMANDA,RATIO_DEMANDA_PARO,PROP_POBLACION_EXTRANJERA,PROP_ENVEJECIMIENTO,INDICE_ATRACCION_EXTERNA
3294,2005,42001 ABEJAR,392,187,205,42001,7.0,13.0,1.0,5.0,...,10.0,7.0,11.0,29.0,2.92887,4.1841,1.428571,0.035714,0.306122,0.551724
3295,2005,42003 ADRADAS,82,37,45,42003,0.0,0.0,0.0,0.0,...,3.0,3.0,0.0,0.0,7.142857,7.142857,1.000000,0.0,0.439024,NaN
3296,2005,42004 AGREDA,3216,1599,1617,42004,24.0,25.0,2.0,2.0,...,100.0,85.0,97.0,83.0,4.372428,5.144033,1.176471,0.047886,0.266169,0.698795
3297,2005,42006 ALCONABA,171,69,102,42006,9.0,1.0,0.0,0.0,...,7.0,4.0,10.0,1.0,3.539823,6.19469,1.750000,0.011696,0.25731,0.000000
3298,2005,42007 ALCUBILLA DE AVELLANEDA,183,93,90,42007,4.0,1.0,3.0,2.0,...,5.0,5.0,9.0,6.0,6.024096,6.024096,1.000000,0.016393,0.519126,0.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3389,2005,42111 MATAMALA DE ALMAZAN,372,174,198,42111,6.0,5.0,2.0,5.0,...,15.0,13.0,12.0,18.0,6.341463,7.317073,1.153846,0.008065,0.389785,0.722222
3390,2005,42113 MEDINACELI,694,345,349,42113,8.0,14.0,10.0,2.0,...,23.0,16.0,45.0,72.0,3.990025,5.735661,1.437500,0.096542,0.31268,0.805556
3391,2005,42115 MIÑO DE MEDINACELI,99,50,49,42115,11.0,1.0,0.0,0.0,...,0.0,0.0,18.0,14.0,0.0,0.0,NaN,0.0,0.363636,0.928571
3392,2005,42116 MIÑO DE SAN ESTEBAN,75,32,43,42116,2.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,NaN,0.0,0.653333,NaN


In [117]:
df_final.columns

Index(['FECHA', 'MUNICIPIO', 'POBLACION_TOTAL', 'POBLACION_MUJERES',
       'POBLACION_HOMBRES', 'COD_MUNICIPIO',
       'EMIGRACIONES CON DESTINO A LA MISMA PROVINCIA',
       'INMIGRACIONES PROCEDENTES DE LA MISMA PROVINCIA',
       'EMIGRACIONES CON DESTINO A OTRA PROVINCIA DE CASTILLA Y LEÓN',
       'INMIGRACIONES PROCEDENTES DE OTRA PROVINCIA DE CASTILLA Y LEÓN',
       'EMIGRACIONES CON DESTINO A OTRAS COMUNIDADES AUTÓNOMAS',
       'INMIGRACIONES PROCEDENTES DE OTRAS COMUNIDADES AUTÓNOMAS',
       'EMIGRACIONES CON DESTINO A OTROS PAÍSES', 'CONSULTORIOS', 'FARMACIAS',
       'HOSPITALES', 'EMPRESAS', 'ALUMNOS', 'ESCUELAS', 'SERVICIOS_TURISTICOS',
       'SERVICIOS_SANIDAD', 'DEMANDAS_EMPLEO', 'PARO', 'EMIGRACIONES_TOTALES',
       'INMIGRACIONES_TOTALES', 'TASA_PARO', 'TASA_DEMANDA',
       'RATIO_DEMANDA_PARO', 'PROP_POBLACION_EXTRANJERA',
       'PROP_ENVEJECIMIENTO', 'INDICE_ATRACCION_EXTERNA'],
      dtype='object')

In [118]:
#Añadimos un fichero más que hemos cargado anteriormente, que es el de la distancia a la capital. lo unimos solo por codigo de municipio. 
df_final = df_final.merge(distancia, on="COD_MUNICIPIO", how="left")

In [119]:
df_final = df_final.rename(
    columns={"distancia en km a la capital (soria)": "DISTANCIA_CAPITAL"}
)
df_final.head()

,FECHA,MUNICIPIO,POBLACION_TOTAL,POBLACION_MUJERES,POBLACION_HOMBRES,COD_MUNICIPIO,EMIGRACIONES CON DESTINO A LA MISMA PROVINCIA,INMIGRACIONES PROCEDENTES DE LA MISMA PROVINCIA,EMIGRACIONES CON DESTINO A OTRA PROVINCIA DE CASTILLA Y LEÓN,INMIGRACIONES PROCEDENTES DE OTRA PROVINCIA DE CASTILLA Y LEÓN,...,PARO,EMIGRACIONES_TOTALES,INMIGRACIONES_TOTALES,TASA_PARO,TASA_DEMANDA,RATIO_DEMANDA_PARO,PROP_POBLACION_EXTRANJERA,PROP_ENVEJECIMIENTO,INDICE_ATRACCION_EXTERNA,DISTANCIA_CAPITAL
0,2005,42001 ABEJAR,392,187,205,42001,7.0,13.0,1.0,5.0,...,7.0,11.0,29.0,2.92887,4.1841,1.428571,0.035714,0.306122,0.551724,27.4
1,2005,42003 ADRADAS,82,37,45,42003,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,7.142857,7.142857,1.000000,0.0,0.439024,NaN,54.3
2,2005,42004 AGREDA,3216,1599,1617,42004,24.0,25.0,2.0,2.0,...,85.0,97.0,83.0,4.372428,5.144033,1.176471,0.047886,0.266169,0.698795,57.1
3,2005,42006 ALCONABA,171,69,102,42006,9.0,1.0,0.0,0.0,...,4.0,10.0,1.0,3.539823,6.19469,1.750000,0.011696,0.25731,0.000000,13.0
4,2005,42007 ALCUBILLA DE AVELLANEDA,183,93,90,42007,4.0,1.0,3.0,2.0,...,5.0,9.0,6.0,6.024096,6.024096,1.000000,0.016393,0.519126,0.833333,78.9


Miramos el tipo de todas las columnas para ver si es el correcto 

In [121]:
df_final.dtypes

FECHA                                                               int32
MUNICIPIO                                                          object
POBLACION_TOTAL                                                     int64
POBLACION_MUJERES                                                   int64
POBLACION_HOMBRES                                                   int64
COD_MUNICIPIO                                                      object
EMIGRACIONES CON DESTINO A LA MISMA PROVINCIA                     float64
INMIGRACIONES PROCEDENTES DE LA MISMA PROVINCIA                   float64
EMIGRACIONES CON DESTINO A OTRA PROVINCIA DE CASTILLA Y LEÓN      float64
INMIGRACIONES PROCEDENTES DE OTRA PROVINCIA DE CASTILLA Y LEÓN    float64
EMIGRACIONES CON DESTINO A OTRAS COMUNIDADES AUTÓNOMAS            float64
INMIGRACIONES PROCEDENTES DE OTRAS COMUNIDADES AUTÓNOMAS          float64
EMIGRACIONES CON DESTINO A OTROS PAÍSES                           float64
CONSULTORIOS                          

Vamos a dividir el dataframe en dos, uno con las variables que queremos utilizar en el modelo, y otro con mas informacion que puede ser relevante para visualizar

In [123]:
cols_modelo = [
    "FECHA",
    "COD_MUNICIPIO",
    "MUNICIPIO",
    "POBLACION_TOTAL",
    "PROP_ENVEJECIMIENTO",
    "TASA_DEMANDA",
    "TASA_PARO",
    "RATIO_DEMANDA_PARO",
    "SERVICIOS_TURISTICOS",
    "SERVICIOS_SANIDAD",
    "CONSULTORIOS",
    "FARMACIAS",
    "HOSPITALES",
    "ESCUELAS",
    "ALUMNOS",
    "EMPRESAS",
    "PROP_POBLACION_EXTRANJERA",
    "DISTANCIA_CAPITAL"
]

df_modelo = df_final[cols_modelo].copy()


In [124]:
cols_visual = [
    "FECHA",
    "COD_MUNICIPIO",
    "MUNICIPIO",
    "POBLACION_MUJERES",
    "POBLACION_HOMBRES",
    "EMIGRACIONES CON DESTINO A LA MISMA PROVINCIA",
    "INMIGRACIONES PROCEDENTES DE LA MISMA PROVINCIA",
    "EMIGRACIONES CON DESTINO A OTRA PROVINCIA DE CASTILLA Y LEÓN",
    "INMIGRACIONES PROCEDENTES DE OTRA PROVINCIA DE CASTILLA Y LEÓN",
    "EMIGRACIONES CON DESTINO A OTRAS COMUNIDADES AUTÓNOMAS",
    "INMIGRACIONES PROCEDENTES DE OTRAS COMUNIDADES AUTÓNOMAS",
    "EMIGRACIONES CON DESTINO A OTROS PAÍSES"
]
df_visual = df_final[cols_visual].copy()

In [125]:
df_modelo.shape
df_modelo.head()

,FECHA,COD_MUNICIPIO,MUNICIPIO,POBLACION_TOTAL,PROP_ENVEJECIMIENTO,TASA_DEMANDA,TASA_PARO,RATIO_DEMANDA_PARO,SERVICIOS_TURISTICOS,SERVICIOS_SANIDAD,CONSULTORIOS,FARMACIAS,HOSPITALES,ESCUELAS,ALUMNOS,EMPRESAS,PROP_POBLACION_EXTRANJERA,DISTANCIA_CAPITAL
0,2005,42001,42001 ABEJAR,392,0.306122,4.1841,2.92887,1.428571,16.0,NaN,NaN,NaN,0.0,NaN,0,NaN,0.035714,27.4
1,2005,42003,42003 ADRADAS,82,0.439024,7.142857,7.142857,1.000000,1.0,NaN,NaN,NaN,0.0,0.0,0,NaN,0.0,54.3
2,2005,42004,42004 AGREDA,3216,0.266169,5.144033,4.372428,1.176471,4.0,NaN,NaN,NaN,0.0,2.0,461,NaN,0.047886,57.1
3,2005,42006,42006 ALCONABA,171,0.25731,6.19469,3.539823,1.750000,0.0,NaN,NaN,NaN,0.0,0.0,0,NaN,0.011696,13.0
4,2005,42007,42007 ALCUBILLA DE AVELLANEDA,183,0.519126,6.024096,6.024096,1.000000,0.0,NaN,NaN,NaN,0.0,0.0,0,NaN,0.016393,78.9


In [126]:
df_visual.shape
df_visual.head()

,FECHA,COD_MUNICIPIO,MUNICIPIO,POBLACION_MUJERES,POBLACION_HOMBRES,EMIGRACIONES CON DESTINO A LA MISMA PROVINCIA,INMIGRACIONES PROCEDENTES DE LA MISMA PROVINCIA,EMIGRACIONES CON DESTINO A OTRA PROVINCIA DE CASTILLA Y LEÓN,INMIGRACIONES PROCEDENTES DE OTRA PROVINCIA DE CASTILLA Y LEÓN,EMIGRACIONES CON DESTINO A OTRAS COMUNIDADES AUTÓNOMAS,INMIGRACIONES PROCEDENTES DE OTRAS COMUNIDADES AUTÓNOMAS,EMIGRACIONES CON DESTINO A OTROS PAÍSES
0,2005,42001,42001 ABEJAR,187,205,7.0,13.0,1.0,5.0,3.0,11.0,0.0
1,2005,42003,42003 ADRADAS,37,45,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2005,42004,42004 AGREDA,1599,1617,24.0,25.0,2.0,2.0,70.0,56.0,1.0
3,2005,42006,42006 ALCONABA,69,102,9.0,1.0,0.0,0.0,1.0,0.0,0.0
4,2005,42007,42007 ALCUBILLA DE AVELLANEDA,93,90,4.0,1.0,3.0,2.0,2.0,3.0,0.0


In [127]:
#df_modelo.to_csv(
 #   "df_modelo.csv",
 #   index=False,
 #   encoding="utf-8"
#)

In [128]:
df_visual.to_csv(
    "df_visual.csv",
    index=False,
    encoding="utf-8"
)